In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Installing necessary packages

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.0 MB/s 
     |████████████████████████████████| 120 kB 38.0 MB/s 
     |████████████████████████████████| 6.6 MB 43.4 MB/s 


In [ ]:
! pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 7.5 MB/s 


In [ ]:
! pip install sentence-splitter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.1 MB/s 


In [ ]:
pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=44329609220c7b3a2b54f7ca8f2573c8e73b341c70b8ed6b5bc0bd103264912e
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [ ]:
! pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 7.0 MB/s 


In [ ]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-8_avp8s_
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-8_avp8s_
     |████████████████████████████████| 50 kB 2.8 MB/s 
     |████████████████████████████████| 85 kB 2.9 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8602 sha256=cc68a9120bd17a1f72a3afb4cd5584d5ba4f80017d5dd1a28b4c0e9c5aebe210
  Stored in directory: /tmp/pip-ephem-wheel-cache-i5hadr8p/wheels/d8/27/d8/b757755a5b0b804c59714fdd8a4e2733c20802f638f17f7d96
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149865 sha256=96db6e1d19f843a0471cc56fd9891176eb3e95c1baa23212f1141cc271fbfc19
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
  Created wheel for 

*italicized text*# Importing libraries

In [ ]:
import pandas as pd
import torch
import csv
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from parrot import Parrot
from torchmetrics.text.rouge import ROUGEScore
from torchmetrics.text.bert import BERTScore
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


In [ ]:
NO_CUDA = False

# Load the data

In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/full_pos.tsv', sep='\t', error_bad_lines = False)

# Data: [https://github.com/yanaiela/num_fh/blob/master/data/identification/processed/full_pos.tsv]

b'Skipping line 987: expected 8 fields, saw 10\nSkipping line 2052: expected 8 fields, saw 9\nSkipping line 7541: expected 8 fields, saw 9\nSkipping line 10353: expected 8 fields, saw 9\nSkipping line 11962: expected 8 fields, saw 9\nSkipping line 12492: expected 8 fields, saw 10\nSkipping line 16463: expected 8 fields, saw 9\nSkipping line 18364: expected 8 fields, saw 9\nSkipping line 20148: expected 8 fields, saw 9\nSkipping line 21767: expected 8 fields, saw 9\nSkipping line 21989: expected 8 fields, saw 9\nSkipping line 24845: expected 8 fields, saw 9\nSkipping line 25181: expected 8 fields, saw 9\nSkipping line 26118: expected 8 fields, saw 9\nSkipping line 28845: expected 8 fields, saw 9\nSkipping line 32687: expected 8 fields, saw 9\nSkipping line 35549: expected 8 fields, saw 9\nSkipping line 39105: expected 8 fields, saw 9\nSkipping line 39834: expected 8 fields, saw 9\nSkipping line 40302: expected 8 fields, saw 9\nSkipping line 43608: expected 8 fields, saw 9\nSkipping line

In [ ]:
data

,id,scene_ind,sentence_ind,text,sentence,target,ind_s,ind_e
0,1,0,0,"I don't know, Harvey. It's just starting to fe...","I mean, I'm 22 and I'm in grad school.",22,5,6.0
1,2,3,1,Until 8 in the morning.,Until 8 in the morning.,8,1,2.0
2,5,1,0,I believe that Andy didn't know I had 2 bottle...,I believe that he switched one out knowing I'd...,one,5,6.0
3,12,2,1,"Man of a thousand faces, every one the same.","Man of a thousand faces, every one the same.",one,7,8.0
4,12,16,1,Singing isn't one of them.,Singing isn't one of them.,one,3,4.0
...,...,...,...,...,...,...,...,...
79427,117633,26,2,You two hooked up yet?,You two hooked up yet?,two,1,2.0
79428,117685,26,0,Are you sure no one else is in the house?,Are you sure no one else is in the house?,one,4,5.0
79429,117766,6,0,It's nearly 2 A.M. and we're still sealed in t...,It's nearly 2 A.M. and we're still sealed in t...,2,3,4.0
79430,117798,24,0,The sound of my shout spreads at 360 metres a ...,The Earth cooled off but it took one or two bi...,one,7,8.0


# Generate sentences using the theree models

In [ ]:
model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.to('cuda')
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')
list_paraphrases = []
for index, row in data[0:300].iterrows():
    batch = tokenizer(row['text'], return_tensors='pt')
    batch = batch.to('cuda')
    generated_ids = model.generate(batch['input_ids'])
    generated_sentence = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    list_paraphrases.append((row['text'], row['target'], generated_sentence))

In [ ]:
header = ['Text', 'Target', 'BART']
with open('/content/gdrive/My Drive/Colab Notebooks/generation1.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(header)
  for i in list_paraphrases:
    writer.writerow(i)

In [ ]:
generated_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/generation1.csv')

In [ ]:
# Pegasus mdoel:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
pegasus_list =[]
for i in generated_data['Text']:
  pegasus_list.append(get_response(i, 1))

generated_data['Pegasus'] = pegasus_list

In [ ]:
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

In [ ]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/686 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
Parrot_list =[]
for i in generated_data['Text']:
  Parrot_list.append(parrot.augment(input_phrase=i))

Token indices sequence length is longer than the specified maximum sequence length for this model (635 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
generated_data['Perrot'] = Parrot_list

In [ ]:
generated_data

,Text,Target,BART,Pegasus,Perrot
0,"I don't know, Harvey. It's just starting to fe...",22,"[""I don't know, Harvey. It's just starting to ...",[It's just starting to feel like we're not on ...,"[(I don't know, Harvey. It's just starting to ..."
1,Until 8 in the morning.,8,['Until 8 a.m. in the morning.'],[Until 8 in the morning.],None
2,I believe that Andy didn't know I had 2 bottle...,one,"[""I believe that Andy didn't know I had 2 bott...",[I believe that Andy switched out one of the b...,[(I believe that Andy didn't know I had 2 bott...
3,"Man of a thousand faces, every one the same.",one,"['Man of a thousand faces, every one the same.']",[A man with a thousand faces.],"[(Man of a thousand faces, every one the same...."
4,Singing isn't one of them.,one,['Singing is not one of them.'],[Singing isn't one of them.],"[(Singing isn't one of them., 0)]"
...,...,...,...,...,...
295,"Wow, okay what about this? Maybe Donald is Tra...",60,"[""Wow, okay what about this? maybe Donald is T...","[Maybe Donald is Tracy's son, because Tracy is...","[(Wow, okay what about this? Maybe Donald is T..."
296,"I'm 42, Cerie.",42,"[""I'm 42, Cerie.""]",[I'm 42 years old.],"[(I'm 42, Cerie., 0)]"
297,"Look, I know she gets under your skin, but you...",87,"['Look, I know she gets under your skin, but y...","[I know she gets under your skin, but you shou...",[(i know it gets under your skin but you shoul...
298,"Thats only 14 in demon years, Lemon.",14,"['Thats only 14 in demon years, Lemon.']","[In demon years, thats only 14.]","[(Thats only 14 in demon years, Lemon., 0)]"


In [ ]:
generated_data.to_csv('/content/gdrive/My Drive/Colab Notebooks/datae300_without context.csv', sep="\t", index=False)

# Take scores

In [ ]:
rouge = ROUGEScore()

In [ ]:
def get_bleu_score(x, y):
  return sentence_bleu(x,y)

generated_data['Bleu_BART'] =  generated_data.apply(lambda x: get_bleu_score(x['Text'], x['BART']) if x['BART'] != None  else print("Not available"), axis=1)
generated_data['Bleu_Pegasus'] = generated_data.apply(lambda x: get_bleu_score(x['Text'], x['Pegasus'][0]) if x['Pegasus'] != None  else print("Not available"), axis=1)
generated_data['Bleu_Perrot'] = generated_data.apply(lambda x: get_bleu_score(x['Text'], x['Perrot'][0][0]) if x['Perrot'] != None  else print("Not available"), axis=1)

Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available


In [ ]:
def get_rouge_score(x, y):
  return rouge(x,y)

generated_data['Rouge_BART'] =  generated_data.apply(lambda x: get_rouge_score(x['Text'], x['BART']) if x['BART'] != None  else print("Not available"), axis=1)
generated_data['Rouge_Pegasus'] = generated_data.apply(lambda x: get_rouge_score(x['Text'], x['Pegasus'][0]) if x['Pegasus'] != None  else print("Not available"), axis=1)
generated_data['Rouge_Perrot'] = generated_data.apply(lambda x: get_rouge_score(x['Text'], x['Perrot'][0][0]) if x['Perrot'] != None  else print("Not available"), axis=1)


Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available
Not available


In [ ]:
generated_data

,Text,Target,BART,Pegasus,Perrot,Blue_BART,Blue_Pegasus,Blue_Perrot,Rouge_BART,Rouge_Pegasus,Rouge_Perrot
0,"I don't know, Harvey. It's just starting to fe...",22,"[""I don't know, Harvey. It's just starting to ...",[It's just starting to feel like we're not on ...,"[(I don't know, Harvey. It's just starting to ...",1.394077e-231,1.423643e-231,1.097444e-231,"{'rouge1_fmeasure': tensor(0.4324), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.2778), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
1,Until 8 in the morning.,8,['Until 8 a.m. in the morning.'],[Until 8 in the morning.],None,1.481675e-231,1.609194e-231,NaN,"{'rouge1_fmeasure': tensor(0.8333), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis...",None
2,I believe that Andy didn't know I had 2 bottle...,one,"[""I believe that Andy didn't know I had 2 bott...",[I believe that Andy switched out one of the b...,[(I believe that Andy didn't know I had 2 bott...,1.418620e-231,1.327671e-231,1.170835e-231,"{'rouge1_fmeasure': tensor(0.5833), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.4898), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
3,"Man of a thousand faces, every one the same.",one,"['Man of a thousand faces, every one the same.']",[A man with a thousand faces.],"[(Man of a thousand faces, every one the same....",1.445062e-231,1.531972e-231,1.476840e-231,"{'rouge1_fmeasure': tensor(1.), 'rouge1_precis...","{'rouge1_fmeasure': tensor(0.5333), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
4,Singing isn't one of them.,one,['Singing is not one of them.'],[Singing isn't one of them.],"[(Singing isn't one of them., 0)]",1.493482e-231,1.560619e-231,1.560619e-231,"{'rouge1_fmeasure': tensor(0.6667), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
...,...,...,...,...,...,...,...,...,...,...,...
295,"Wow, okay what about this? Maybe Donald is Tra...",60,"[""Wow, okay what about this? maybe Donald is T...","[Maybe Donald is Tracy's son, because Tracy is...","[(Wow, okay what about this? Maybe Donald is T...",1.353754e-231,1.491300e-231,1.404929e-231,"{'rouge1_fmeasure': tensor(0.9677), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.7692), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
296,"I'm 42, Cerie.",42,"[""I'm 42, Cerie.""]",[I'm 42 years old.],"[(I'm 42, Cerie., 0)]",1.646211e-231,1.554020e-231,1.752958e-231,"{'rouge1_fmeasure': tensor(1.), 'rouge1_precis...","{'rouge1_fmeasure': tensor(0.6667), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
297,"Look, I know she gets under your skin, but you...",87,"['Look, I know she gets under your skin, but y...","[I know she gets under your skin, but you shou...",[(i know it gets under your skin but you shoul...,1.333870e-231,1.356868e-231,1.325250e-231,"{'rouge1_fmeasure': tensor(0.9143), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.8824), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.8235), 'rouge1_pr..."
298,"Thats only 14 in demon years, Lemon.",14,"['Thats only 14 in demon years, Lemon.']","[In demon years, thats only 14.]","[(Thats only 14 in demon years, Lemon., 0)]",1.531972e-231,1.580666e-231,1.572860e-231,"{'rouge1_fmeasure': tensor(1.), 'rouge1_precis...","{'rouge1_fmeasure': tensor(0.9231), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."


In [ ]:
generated_data.to_csv('/content/gdrive/My Drive/Colab Notebooks/data300_without_context_with_scores.csv', sep="\t", index=False)


In [ ]:
def get_bleu_rouge_score(data, column_list):
  scores_bleu = {}
  scores_rouge = {}
  sentences = []
  gen_sentences = []
  for i in column_list:
    for index, row in data.iterrows():
      sentences.append(row['Text'])
      gen_sentences.append(row[i])
    bleu_score= corpus_bleu(sentences, gen_sentences)
    rouge_score= rouge(sentences, gen_sentences)
    scores_bleu['bleu score for {}'.format(i)] = bleu_score
    scores_rouge['rouge score for {}'.format(i)] = rouge_score
  return(scores_bleu, scores_rouge)

In [ ]:
list = ['BART', 'Pegasus'] #perrot

In [ ]:
print(get_bleu_rouge_score(generated_data, list))

({'blue score for BART': 1.3791102692546092e-231, 'blue score for Pegasus': 1.3739893416525898e-231}, {'rouge score for BART': {'rouge1_fmeasure': tensor(0.6846), 'rouge1_precision': tensor(0.6037), 'rouge1_recall': tensor(0.9764), 'rouge2_fmeasure': tensor(0.6426), 'rouge2_precision': tensor(0.5640), 'rouge2_recall': tensor(0.9366), 'rougeL_fmeasure': tensor(0.6813), 'rougeL_precision': tensor(0.6005), 'rougeL_recall': tensor(0.9719), 'rougeLsum_fmeasure': tensor(0.6817), 'rougeLsum_precision': tensor(0.6007), 'rougeLsum_recall': tensor(0.9732)}, 'rouge score for Pegasus': {'rouge1_fmeasure': tensor(0.3241), 'rouge1_precision': tensor(0.2596), 'rouge1_recall': tensor(0.8467), 'rouge2_fmeasure': tensor(0.2257), 'rouge2_precision': tensor(0.1868), 'rouge2_recall': tensor(0.3559), 'rougeL_fmeasure': tensor(0.2972), 'rougeL_precision': tensor(0.2376), 'rougeL_recall': tensor(0.8022), 'rougeLsum_fmeasure': tensor(0.3056), 'rougeLsum_precision': tensor(0.2439), 'rougeLsum_recall': tensor(0.

In [ ]:
def get_bleu_rouge_score(data, column_list ):
  sentences = []
  gen_sentences = []
  for i in column_list:
    for index, row in data.iterrows():
      sentences.append(row['Text'])
      if row[i] != None:
        gen_sentences.append(str(row[i]))
  bleu_score= corpus_bleu(sentences, gen_sentences,  weights=(0.25, 0.25, 0.25, 0.25),  smoothing_function=None,
    auto_reweigh=False)
  score_rouge = rouge(sentences, gen_sentences)
  assert len(sentences) == len(gen_sentences), (
         "The number of hypotheses and their reference(s) should be the same ")
  return('bleu_score: ', bleu_score, 'rouge_score: ', score_rouge)

In [ ]:
list2 = ['Perrot']

In [ ]:
print(get_bleu_rouge_score(generated_data,list2 ))

('bleu_score: ', 9.69543395326178e-232, 'rouge_score: ', {'rouge1_fmeasure': tensor(0.6557), 'rouge1_precision': tensor(0.7881), 'rouge1_recall': tensor(0.6300), 'rouge2_fmeasure': tensor(0.6143), 'rouge2_precision': tensor(0.7385), 'rouge2_recall': tensor(0.5905), 'rougeL_fmeasure': tensor(0.6488), 'rougeL_precision': tensor(0.7786), 'rougeL_recall': tensor(0.6236), 'rougeLsum_fmeasure': tensor(0.6502), 'rougeLsum_precision': tensor(0.7803), 'rougeLsum_recall': tensor(0.6250)})


In [ ]:
generated_data['Rouge_BART'][0]

{'rouge1_fmeasure': tensor(0.4324),
 'rouge1_precision': tensor(0.2759),
 'rouge1_recall': tensor(1.),
 'rouge2_fmeasure': tensor(0.4167),
 'rouge2_precision': tensor(0.2632),
 'rouge2_recall': tensor(1.),
 'rougeL_fmeasure': tensor(0.4324),
 'rougeL_precision': tensor(0.2759),
 'rougeL_recall': tensor(1.),
 'rougeLsum_fmeasure': tensor(0.4324),
 'rougeLsum_precision': tensor(0.2759),
 'rougeLsum_recall': tensor(1.)}

In [ ]:
generated_data.columns

Index(['Text', 'Target', 'BART', 'Pegasus', 'Perrot', 'Blue_BART',
       'Blue_Pegasus', 'Blue_Perrot', 'Rouge_BART', 'Rouge_Pegasus',
       'Rouge_Perrot'],
      dtype='object')

# Random_sample

In [ ]:
data4 = generated_data.sample(n=30, random_state=1)

In [ ]:
data4.to_csv('/content/gdrive/My Drive/Colab Notebooks/data_sample_1.csv', sep="\t", index=False)

# Check equality and difference of generated texts and the original text column

In [ ]:
data4 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data_sample_1.csv', sep='\t', error_bad_lines = False)
data4.head(5)

NameError: ignored

In [ ]:
print(data4['Text'].equals(data4['BART']))

In [ ]:
data4['Target']

In [ ]:
list1 = []
for index, row in data4.iterrows():
    list1.append((row['Target'] in row['BART']))

list1

In [ ]:
data4['isin_BART'] = list1
data4.head()

In [ ]:
data4['New_BART']  = [int(x[0] in x[1]) for x in zip(data4['Text'], data4['BART'])]
data4

In [ ]:
data4['is_equal_Pegasus']= (data4['Text']==data4['Pegasus'])
data4

# Check if the FHs are resolved in the generated text

In [ ]:
string = 'I am writing a text here'

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def give_next_token(token, s):
  tokens = word_tokenize(s)
  index = tokens.index(token)
  next_token = tokens [index +1]
  return next_token

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
give_next_token( "a", string)

'text'

In [ ]:
list_tokens =[]
for index, row in data4.iterrows():
    string2 = row['BART']
    print(string2)
    token2 = row['Target']
    print(token2)
    if token2 in string2:
      print(token2 in string2)
      print('NEXT TOKEN', give_next_token(token2, string2))

["Rose has it in her lease: Two perfect 10's can't share the same apartment."]
10
True
NEXT TOKEN 's
["C'mon, Oleg, I mean, nobody can say for sure that Madonna doesn"]
one
["You hope I don't already have one? Make this go away."]
one
True
NEXT TOKEN ?
["Yeah that's the way Tony sees it too. It didn't happen exactly like that."]
one
['Staufman, as brutal as these times are, there still must be some rules.']
One
["I'm just still on a high from this $10,000 bank loan I got last"]
2004
["The kids name is Reggie Brooks. Guys like him don't handle being humiliated. They lose"]
eighteen
["If I were forced to bet my life on a mission, I'd prefer it would be"]
one
['1, 2, 3, never!']
1
True
NEXT TOKEN ,
['Three, eight, four, six, nine, Romeo. Three, alpha, four']
Three
True
NEXT TOKEN ,
["Kenneth, since you've left the program, finding you a new position here has been"]
one
['You mention the Navy, for example, and that we have fewer ships than we had in']
1916
["So it's really that bad out the

ValueError: ignored